In [2]:
import geopandas as gpd
import contextily as ctx
import matplotlib.pyplot as plt
from cartiflette import carti_download

In [ ]:
communes_borders = carti_download(
    crs = 4326,
    values = ["75", "92", "93", "94"],
    borders="COMMUNE",
    vectorfile_format="geojson",
    filter_by="DEPARTEMENT",
    source="EXPRESS-COG-CARTO-TERRITOIRE",
    year=2022)

arrondissements_borders = carti_download(
    crs = 4326,
    values = ["75", "92", "93", "94"],
    borders="COMMUNE_ARRONDISSEMENT",
    vectorfile_format="geojson",
    filter_by="DEPARTEMENT",
    source="EXPRESS-COG-CARTO-TERRITOIRE",
    year=2022)

In [ ]:
print(communes_borders.columns)
print(communes_borders.crs)

communes_borders = communes_borders.to_crs(2154)
communes_borders.crs

hauts_de_seine = communes_borders[communes_borders['INSEE_DEP']=='92']
hauts_de_seine.plot(edgecolor='black', facecolor='None')

In [ ]:
paris = communes_borders[communes_borders['INSEE_DEP']=='75']
paris.plot(edgecolor='black', facecolor='None')

paris2 = arrondissements_borders[arrondissements_borders['INSEE_DEP']=='75']
paris2.plot(facecolor='None', edgecolor='black')

In [ ]:
idf = carti_download(
      values = ["11"],
      crs = 4326,
      borders = "DEPARTEMENT",
      vectorfile_format="geojson",
      filter_by="REGION",
      source="EXPRESS-COG-CARTO-TERRITOIRE",
      year=2022)

petite_couronne_departements = idf[idf['INSEE_DEP'].isin(["75","92","93","94"])].to_crs(2154)

: 